## 10th Circuit Court Opinion Data Bibliography

#### Overview

The data is  from [Court Listener](https://www.courtlistener.com), a Free Law Project...project. [Free Law Project](https://free.law) is a data/software oriented nonprofit seeking to "make the legal ecosystem more equitable and competitive." The [Court Listener API](https://www.courtlistener.com/api/) allows for bulk dowloads of court proceedings including arguments and opinions from each Circuit Court and the Supreme Court. I found the Court Listener API this [this tutorial](https://github.com/idc9/word_embed_tutorial) which creates a word embedding with Supreme Court opinions.

Note: put the opinions from [here](https://www.courtlistener.com/api/bulk-data/opinions/ca10.tar.gz) in the same directory and name directory 'ca10'

### Contents

In [4]:
import glob
import os
json_files = glob.glob('/Users/keira.richards/E_DHML/embed/ca10/*' + "*.json")
len(json_files)

56194

The data set is 56194 JSON files of court opinions from the 10th circuit court. Each file is one opinion on one case. The cases are chornological, but the numbered files arent, alluding to possible missing cases. it begins with 95.json from march 13, 2010 and ends with 6216474.json from Febuary 8, 2022. The data does not go as far back as the Supreme Court opinions available on Court Listiner, but will lend itself well to analyzing change over the last decade. I'm going to read in the first file to look at the contents qualitatively 

In [5]:
op95= open('ca10/95.json').read()
op95[:3000]

'{\n  "resource_uri": "https://www.courtlistener.com/api/rest/v3/opinions/95/",\n  "id": 95,\n  "absolute_url": "/opinion/95/united-states-v-frownfelter/",\n  "cluster": "https://www.courtlistener.com/api/rest/v3/clusters/95/",\n  "author": null,\n  "joined_by": [],\n  "author_str": "",\n  "per_curiam": false,\n  "joined_by_str": "",\n  "date_created": "2010-03-13T23:59:33Z",\n  "date_modified": "2020-03-02T19:53:50.186931Z",\n  "type": "010combined",\n  "sha1": "ee78eebf5c1aaa2cfa3f382c1b364e6573f0067c",\n  "page_count": 7,\n  "download_url": "http://www.ca10.uscourts.gov/opinions/10/10-4016.pdf",\n  "local_path": "pdf/2010/03/11/United_States_vs._Frownfelter_1.pdf",\n  "plain_text": "                                                                        FILED\\n                                                             United States Court of Appeals\\n                                                                     Tenth Circuit\\n\\n                                           

Ew, Lets see if i can make it more readable

In [6]:
import re
op95_readable = re.split(r"\n|\\n", op95)
op95_readable[:50]

['{',
 '  "resource_uri": "https://www.courtlistener.com/api/rest/v3/opinions/95/",',
 '  "id": 95,',
 '  "absolute_url": "/opinion/95/united-states-v-frownfelter/",',
 '  "cluster": "https://www.courtlistener.com/api/rest/v3/clusters/95/",',
 '  "author": null,',
 '  "joined_by": [],',
 '  "author_str": "",',
 '  "per_curiam": false,',
 '  "joined_by_str": "",',
 '  "date_created": "2010-03-13T23:59:33Z",',
 '  "date_modified": "2020-03-02T19:53:50.186931Z",',
 '  "type": "010combined",',
 '  "sha1": "ee78eebf5c1aaa2cfa3f382c1b364e6573f0067c",',
 '  "page_count": 7,',
 '  "download_url": "http://www.ca10.uscourts.gov/opinions/10/10-4016.pdf",',
 '  "local_path": "pdf/2010/03/11/United_States_vs._Frownfelter_1.pdf",',
 '  "plain_text": "                                                                        FILED',
 '                                                             United States Court of Appeals',
 '                                                                     Tenth 

In [7]:
#uncomment the following to read the whole thing
#print(op95_readable)

Good enough, and look at all that helpful metadata at the top. Reading it as a human, the files contain a breif summary of the case, the decision of the court and the rational behind the decision. The author meta for all the files ones ive looked at are blank, but in the files they all seem to be written by respective "clerks of the court". That makes sense given their job is record keeping, but its hard to tell if this was what the judge said in court or was dictated later. I notice in the media it says '"per curiam" = flase' meaning its not a judges decison. I think i will try to figure out if i can narrow down the opinions to just per curiam opinions using the meta data

In [56]:
ca10='ca10/'
for file_name in os.listdir(ca10):
    file=os.path.join(ca10,file_name)
    txt=open(file).read()
    split=re.split(r"\n|\\n", txt)
    string=split[8]
    if 'true' in string:
        shutil.copyfile(file,'per_curiam')

KeyboardInterrupt: 

Not working at the moment but lets try pulling dates. the date can be found easily enough in the 'readable' version....

In [50]:
lit=op95_readable[10]
print(lit[-22:-12])

2010-03-13


...Lets see if i can make a loop that puts the dates in a list

In [9]:
dates=[]
ca10='/Users/keira.richards/E_DHML/embed/ca10/'
for file_name in os.listdir(ca10):
    file=os.path.join(ca10,file_name)
    txt=open(file).read()
    split=re.split(r"\n|\\n", txt)
    date_string=split[10]
    date_actual= date_string[-22:-12:]
    dates.append(date_actual)
len(dates)

KeyboardInterrupt: 

That kind of worked...when I run a random sample we can see it isnt perfectly pulling dates and there is one extra (???), but I'll come back to that later.


In [ ]:
import random
random.sample(dates, 15) #that aint right, fix it fucker

Found someone with a similar issue on stack overflow, going to try to adapt their function to extracting meta from my files (have yet to get this to run with the processing issues

In [34]:
import json
import pandas as pd


level = 4
df = pd.DataFrame(columns = ["key0", "dt0", "key1", "dt1", "key2", "dt2", "key3", "dt3", "key4", "dt4"])
def parse_json(key, value,curent_level,massage):
    global df
    global level
    if curent_level == level:
        df = df.append(massage,ignore_index = True)
        return
    t = type(value)
    if t == str:
        try:
            tmp = json.loads(value)
            massage["key"+str(curent_level)] = key
            massage["dt"+str(curent_level)] = "json"
            for k in tmp:
                parse_json(k,tmp[k],curent_level+1,massage)
        except:
            massage["key"+str(curent_level)] = key
            massage["dt"+str(curent_level)] = "str"
            df = df.append(massage,ignore_index = True)
            return
    else:
        if t == list:
            massage["key"+str(curent_level)] = key
            massage["dt"+str(curent_level)] = "list"
            for li in value:
                parse_json(key,li,curent_level+1,massage)
        if t == dict:
            massage["key"+str(curent_level)] = key
            massage["dt"+str(curent_level)] = "dict"
            for k in value:
                parse_json(k,value[k],curent_level+1,massage)
        if t == set:
            massage["key"+str(curent_level)] = key
            massage["dt"+str(curent_level)] = "set"
            for li in value:
                parse_json(key,li,curent_level+1,massage)

        massage["key"+str(curent_level)] = key
        massage["dt"+str(curent_level)] = "simple"
        df = df.append(massage,ignore_index = True)

ca10='/Users/keira.richards/E_DHML/embed/ca10/'
for file_name in os.listdir(ca10):
    path=os.path.join(ca10,file_name)
    file = open(path)
    json_file = json.load(file)
    parse_json("json_file", json_file,0,{})
    df.to_excel(r"out.xlsx")

KeyboardInterrupt: 

### Non-ML Analyis
Im going to try to do some term frequency analayis on a random sample of the Json files. After muplitplee attempts to use the whole set, I dont think my 2015 Mac has the RAM. But he tried and im proud of him.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from pathlib import Path  

In [22]:
json_sample=random.sample (json_files, 5000)
titles = [Path(text).stem for text in json_sample]

In [23]:
tfidf_vectorizer = TfidfVectorizer(input='filename', stop_words='english')
tfidf_vector = tfidf_vectorizer.fit_transform(json_sample)

In [24]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=titles, columns=tfidf_vectorizer.get_feature_names())

In [26]:
tfidf_df.loc['00_Document Frequency'] = (tfidf_df > 0).sum()
tfidf_slice = tfidf_df[['government', 'men', 'man', 'women', 'woman', 'race','rights', 'children']]
tfidf_slice.sort_index().round(decimals=2)

,government,men,man,women,woman,race,rights,children
00_Document Frequency,2025.00,281.0,401.00,136.0,159.00,243.00,2193.00,303.0
1033269,0.02,0.0,0.00,0.0,0.00,0.00,0.00,0.0
1033272,0.00,0.0,0.00,0.0,0.01,0.00,0.02,0.0
1033797,0.00,0.0,0.01,0.0,0.00,0.01,0.01,0.0
1034152,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...
950558,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0
981,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0
984810,0.00,0.0,0.00,0.0,0.00,0.00,0.01,0.0
985209,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0


In [ ]:
top_tfidf = tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

In [29]:
tfidf_df.stack().reset_index()
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})

KeyboardInterrupt: 

In [ ]:
!pip install altair
import altair as alt
import numpy as np

# Terms in this list will get a red dot in the visualization
term_list = ['war', 'peace']

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'document:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["document"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')        
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 600)